Done
- created step plot functionality
- created artificial endpoint generation
- seperated legend label aruguments form hover label arguments
- improved legend generation
- added incomplete target functionality
- created reference implementation for: (1) target distribution monitoring (2) realised performance monitoring (3) correct naming of plotting elements 

Todo
- ordered function arguments by importance and added typing information
- cleaned up the code

In [ ]:
# add incomplete target data functionality
# clean up all arguments

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import nannyml as nml

In [ ]:
from nannyml.plots._step_plot import _step_plot

In [ ]:
reference, analysis, analysis_target = nml.load_synthetic_binary_classification_dataset()
chunk_size = 5000
reference_and_analysis = pd.concat([reference, analysis], ignore_index=True)

In [ ]:
metadata = nml.extract_metadata(data = reference, model_name='wfh_predictor')
metadata.target_column_name = 'work_home_actual'

In [ ]:
CHUNK_KEY_COLUMN_NAME = 'key'

# Confidence based performance estimation

In [ ]:
estimator = nml.CBPE(model_metadata=metadata, chunk_size=chunk_size)
estimator.fit(reference)
estimated_performance = estimator.estimate(data=reference_and_analysis)

In [ ]:
estimation_results = estimated_performance.data

still a bug here, roc_auc in referene period should not be estimated

suggestion: add realised_roc_auc column to output ==> thant that column can be combined with the estimated roc_auc_column to create the metric_column and estimated can be set to false in reference

In [ ]:
estimation_results['thresholds'] = list(zip(estimation_results.lower_threshold, estimation_results.upper_threshold))

estimation_results['estimated'] = True

plot_partition_separator = len(estimation_results.value_counts()) > 1

fig = _step_plot(
    table=estimation_results,
    metric_column_name='estimated_roc_auc',
    chunk_column_name=CHUNK_KEY_COLUMN_NAME,
    drift_column_name='alert',

    threshold_column_name='thresholds',
    threshold_legend_label='Performance threshold',
    title='ROC AUC over time (realised and estimated)',
    y_axis_title='ROC AUC',
    v_line_separating_analysis_period=plot_partition_separator,
    estimated_column_name='estimated',
    confidence_column_name='confidence',
    hover_labels=['Chunk', 'ROC AUC'],
#     hover_marker_labels=['Reference', 'No performance drop', 'Probable performance drop'],
    hover_marker_labels=['', '', ''],
    chunk_legend_labels=['Reference period (realised performance)', 'Analysis period (estimated performance, using CBPE)'],
    confidence_legend_label='Estimated performance confidence band',
    drift_legend_label='Probable drop in performance',
    
)

fig.show()

In [ ]:
import plotly.graph_objects as go
import numpy as np

# data = pd.DataFrame(
#     {'end_date': pd.date_range('2021-01-01', '2021-08-30', freq='M'),
#      'chunk_type': ['p1'] * 3 + ['p2'] * 4,
#      'metric':np.arange(7)}
# )

# data.insert(0, 'start_date', data['end_date'] - pd.offsets.MonthBegin())

# data['mid_point_date'] = data['start_date'] + (data['end_date'] - data['start_date']) / 2

# def _create_artificial_end_point(data):
#     data_point_hack = data.tail(1).copy()
#     data_point_hack['start_date'] = data_point_hack['end_date']
#     data_point_hack['end_date'] = pd.NaT
#     data_point_hack['mid_point_date'] = pd.NaT
#     data_point_hack.index = data_point_hack.index + 1
#     return pd.concat([data, data_point_hack], axis=0)


# fig = go.Figure()

# colors = ['blue', 'red']
# chunk_types = data['chunk_type'].unique()

# for i, chunk_type in enumerate(chunk_types):
    
#     data_subset = create_artificial_end_point(data.loc[(data['chunk_type'] == chunk_type)])
    
#     display(data_subset)
    
#     fig.add_trace(
#         go.Scatter(
#             mode='lines',
#             line=dict(shape='hv', color=colors[i]),
#             x=data_subset['start_date'],
#             y=data_subset['metric'],
#             hoverinfo='skip'
#         )
#     )
    
#     fig.add_trace(
#         go.Scatter(
#             mode='markers',
#             marker=dict(color=colors[i]),
#             x=data_subset['mid_point_date'],
#             y=data_subset['metric']
#         )
#     )
    
# fig

# Reconstruction error

In [ ]:
rcerror_calculator = nml.DataReconstructionDriftCalculator(model_metadata=metadata, chunk_size=chunk_size)
rcerror_calculator.fit(reference_data=reference)
rcerror_results = rcerror_calculator.calculate(data=reference_and_analysis)

In [ ]:
data = rcerror_results.data

In [ ]:
plot_partition_separator = len(data.value_counts()) > 1
data['thresholds'] = list(zip(data.lower_threshold, data.upper_threshold))
fig = _step_plot(
    table=data,
    metric_column_name='reconstruction_error',
    chunk_column_name=CHUNK_KEY_COLUMN_NAME,
    drift_column_name='alert',
    threshold_column_name='thresholds',
    title='Reconstruction error over time',
    y_axis_title='Reconstruction error',
    v_line_separating_analysis_period=plot_partition_separator,
    hover_labels=['Chunk', 'RC Error'],
    hover_marker_labels=['', '', '']
)

fig.show()

# Continuous/cartegoric univariate data drift

In [ ]:
univariate_calculator = nml.UnivariateStatisticalDriftCalculator(model_metadata=metadata, chunk_size=chunk_size)
univariate_calculator.fit(reference_data=reference)
univariate_results = univariate_calculator.calculate(data=reference_and_analysis)

In [ ]:
data = univariate_results.data
metric_column_name, _, drift_column_name, _ = [c for c in univariate_results.data.columns if c.startswith('tenure')]
metric_label = metric_column_name.split('_')[0]

In [ ]:
plot_partition_separator = len(data.value_counts()) > 1

fig = _step_plot(
    table=data,
    metric_column_name=metric_column_name,
    chunk_column_name=CHUNK_KEY_COLUMN_NAME,
    drift_column_name=drift_column_name,
#     threshold_column_name=threshold_column_name,
    title='KS D-statistic of {} over time'.format(metric_label),
    y_axis_title='D-statistic',
    v_line_separating_analysis_period=plot_partition_separator,
    statistically_significant_column_name=drift_column_name,
    hover_labels=['Chunk', 'D-statistic'],
    hover_marker_labels=['', '', '']
)

fig.show()

# Realised performance monitoring

In [ ]:
rperf_results = estimation_results.copy()
rperf_results = rperf_results.rename(columns={'estimated_roc_auc':'roc_auc'})
rperf_results = rperf_results.drop(columns=['confidence', 'estimated'])

In [ ]:
rperf_results['realised_target_percentage'] = 1
rperf_results.iloc[-8:, -1] = [0.33] * 8

when realised target percange < 0.25, rows should be removed 

! use with caution, as it assumed incompletion rate is monotonous decreasing from the moment it happens

See example in target distribution monitoring

In [ ]:
plot_partition_separator = len(estimation_results.value_counts()) > 1

fig = _step_plot(
    table=rperf_results,
    metric_column_name='roc_auc',
    chunk_column_name=CHUNK_KEY_COLUMN_NAME,
    drift_column_name='alert',
    threshold_column_name='thresholds',
    threshold_legend_label='Performance threshold',
    title='ROC AUC over time (realised)',
    y_axis_title='ROC AUC',
    v_line_separating_analysis_period=plot_partition_separator,
    hover_labels=['Chunk', 'ROC AUC', 'Target data'],
#     hover_marker_labels=['Reference', 'No performance change', 'Performance change'],
    hover_marker_labels=['', '', ''],
    drift_legend_label='Change in performance',
    partial_target_column_name='realised_target_percentage'
    
)

fig.show()

# Target distribution monitoring

In [ ]:
tdist_results = estimation_results.copy()
tdist_results = tdist_results.drop(columns=['confidence', 'estimated'])
tdist_results = tdist_results.rename(columns={'estimated_roc_auc':'churn_rate'})
tdist_results['churn_rate'] = tdist_results['churn_rate'] - 0.9

tdist_results.lower_threshold = tdist_results.lower_threshold - 0.9
tdist_results.upper_threshold = tdist_results.upper_threshold - 0.9
tdist_results['thresholds'] = list(zip(tdist_results.lower_threshold, tdist_results.upper_threshold))

In [ ]:
tdist_results['realised_target_percentage'] = 1
tdist_results.iloc[-3:, -1] = [0.75, 0.33, 0.1]

when realised target percange < 0.25, rows should be removed 

! use with caution, as it assumed incompletion rate is monotonous decreasing from the mome

In [ ]:
tdist_results = tdist_results.loc[tdist_results['realised_target_percentage'] > 0.25, ]

In [ ]:
plot_partition_separator = len(estimation_results.value_counts()) > 1

fig = _step_plot(
    table=tdist_results,
    metric_column_name='churn_rate',
    chunk_column_name=CHUNK_KEY_COLUMN_NAME,
    drift_column_name='alert',
    statistically_significant_column_name='alert',
    drift_legend_label='Target drift',
    threshold_column_name='thresholds',
    threshold_legend_label='Target threshold',
    title='Churn rate over time',
    y_axis_title='Churn rate',
    v_line_separating_analysis_period=plot_partition_separator,
    hover_labels=['Chunk', 'Churn rate', 'Target data'],
#     hover_marker_labels=['Reference', 'No target drift', 'Target drift'],
    hover_marker_labels=['', '', ''],
    partial_target_column_name='realised_target_percentage'
)

fig.show()